In [ ]:
import torch
from genesis.factories.datasets import DatasetFactory,DataLoaderFactory
from genesis.models import vanilla_autoencoder as models
from genesis.data_fetchers import data_fetcher as fetcher
from genesis.configs import config
from genesis.trainers import neural_net as trainer

experiment_name = "vanilla_vae_experiment"
epochs = 100

print(f"Initializing experiment: {experiment_name}")

# load configs
data_config = config.DataConfig()
data_loader_config = config.DataLoaderConfig()




# data initialization
print("Step 1: Fetching and organizing data...")
fetcher = fetcher.KaggleFetcher(data_config)
data_path = fetcher.fetch(reorganize=True)

# Step 2: Create data_fetchers
print("\nStep 2: Creating data_fetchers...")
dataset_factory = DatasetFactory(
    data_path=data_path,
)

datasets_dict = dataset_factory.create_datasets()

# Step 3: Create dataloaders
print("\nStep 3: Creating dataloaders...")
dataloader_factory = DataLoaderFactory(
    batch_size=1000,
    num_workers=0
)
dataloaders = dataloader_factory.create_dataloaders(datasets_dict)

# Get number of classes
num_classes = dataset_factory.get_num_classes()

print("\n" + "=" * 50)
print("DATA PIPELINE SETUP COMPLETE")
print("=" * 50)
print(f"Number of classes: {num_classes}")
print(f"Dataloaders created: {list(dataloaders.keys())}")

# model initialization
model = models.VanillaVAE()
optimizer = torch.optim.Adam(model.parameters())
scheduler = None
loss_fn = torch.nn.MSELoss()

# training initialization
print(f"Initializing training..")
trainer = trainer.Trainer(experiment_name=experiment_name,
                          model=model,
                          optimizer=optimizer,
                          dataloaders=dataloaders,
                          scheduler=scheduler,
                          loss_fn=loss_fn
                          )

trainer.run(epochs=epochs, validation_step=5, visualization_step=10)




